# Topic Modeling of Amazon Reviews

### Steps:

- Find asin (unique id for a product on Amazon) for all cell phones that belong to the brand "Samsung" 
    -  Filter rows from df_metadata where categories like ['Cell Phones & Accessories', 'Cell Phones','....']
    -  Filter rows where brand = "Samsung"
- Create a new dataframe by joining the asin in the dataframe derived in the previous step with df_reviews to capture all reviews related to Samsung Cell Phones. 

#### Pre-processing:
- Choose the right corpus: 
    -  Choose a phone with maximum number of reviews. 50,000 - 100,000 words. 
- Remove words in the stoplist:
    -  English stopwords
    -  Common/Expected words 
        -  "phone"
        -  "Samsung"
- Lemmatize
- Filter datasets for:
    -  Nouns
    -  Adjectives
- Build a tf-idf matrix 

#### Modeling
- Build topic models using gensim

Import libraries, define functions

In [1]:
import json
import os
import glob
import numpy as np
import random
import pandas as pd
import gzip

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

Read data

In [218]:
# data_dir = '/Users/birupakhya/Documents/Projects/topic modeling/'
df_reviews = getDF(data_dir+'reviews_Cell_Phones_and_Accessories_5.json.gz')
# df_metadata = getDF(data_dir+'samsung_metadata.json.gz')
df_metadata = getDF(data_dir+'meta_Cell_Phones_and_Accessories.json.gz')

In [219]:
df_reviews.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A30TL5EWN6DFXT,120401325X,christina,"[0, 0]",They look good and stick good! I just don't li...,4.0,Looks Good,1400630400,"05 21, 2014"
1,ASY55RVNIL0UD,120401325X,emily l.,"[0, 0]",These stickers work like the review says they ...,5.0,Really great product.,1389657600,"01 14, 2014"
2,A2TMXE2AFO7ONB,120401325X,Erica,"[0, 0]",These are awesome and make my phone look so st...,5.0,LOVE LOVE LOVE,1403740800,"06 26, 2014"
3,AWJ0WZQYMYFQ4,120401325X,JM,"[4, 4]",Item arrived in great time and was in perfect ...,4.0,Cute!,1382313600,"10 21, 2013"
4,ATX7CZYFXI1KW,120401325X,patrice m rogoza,"[2, 3]","awesome! stays on, and looks great. can be use...",5.0,leopard home button sticker for iphone 4s,1359849600,"02 3, 2013"


In [7]:
df_metadata.head(2)

,asin,related,title,price,salesRank,imUrl,brand,categories,description
0,0110400550,"{'also_bought': ['B00C56IXFG', 'B008ZUQWOK', '...",Pink &amp; White 3d Melt Ice-cream Skin Hard C...,3.33,{'Cell Phones & Accessories': 83460},http://ecx.images-amazon.com/images/I/31zn6SOL...,,"[[Cell Phones & Accessories, Cases, Basic Cases]]",Pink & White 3D Melt Ice-Cream Skin Hard Case ...
1,011040047X,"{'buy_after_viewing': ['B008RU7UL2', 'B00698LY...",Purple Hard Case Cover for Iphone 4 4s 4g with...,1.94,{'Cell Phones & Accessories': 495795},http://ecx.images-amazon.com/images/I/41WCZc2d...,NaN,"[[Cell Phones & Accessories, Cases, Basic Cases]]",Purple Hard Case Cover for iPhone 4 4S 4G With...


In [9]:
df_metadata[df_metadata['brand'] == 'Samsung'].head(2)

,asin,related,title,price,salesRank,imUrl,brand,categories,description
103,549980088X,{'buy_after_viewing': ['B001BRHXA6']},Red LED Car/Vehicle Charger for Samsung&copy; ...,3.84,{'Cell Phones & Accessories': 291518},http://ecx.images-amazon.com/images/I/41H0DdRZ...,Samsung,"[[Cell Phones & Accessories, Accessories, Char...",Portable lightweight red led car charger offer...
104,5499800650,NaN,Car Charger for Bell Samsung&copy; Instinct Om...,3.88,{'Cell Phones & Accessories': 917469},http://ecx.images-amazon.com/images/I/41RgM6dR...,Samsung,"[[Cell Phones & Accessories, Accessories, Char...",Car Charger for Bell Samsung Instinct Omnia i9...


In [173]:
cat_cell_phones = [j for i in df_metadata['categories'] for j in i if 'Cell Phones' in j]
cat = [i[0] for i in df_metadata['categories']]
# df_metadata['cat'] = cat
print(len(df_metadata))
print(len(cat))

cat[:10]

346793
346793


[['Cell Phones & Accessories', 'Cases', 'Basic Cases'],
 ['Cell Phones & Accessories', 'Cases', 'Basic Cases'],
 ['Cell Phones & Accessories', 'Cases', 'Basic Cases'],
 ['Cell Phones & Accessories', 'Cases', 'Basic Cases'],
 ['Cell Phones & Accessories', 'Cases', 'Basic Cases'],
 ['Cell Phones & Accessories', 'Cases', 'Basic Cases'],
 ['Cell Phones & Accessories', 'Cases', 'Basic Cases'],
 ['Cell Phones & Accessories', 'Cases', 'Basic Cases'],
 ['Cell Phones & Accessories', 'Cases', 'Basic Cases'],
 ['Cell Phones & Accessories', 'Accessories', 'Data Cables']]